### Flask应用中定义路由
* 通过装饰器定义路由
* 通过app.add_url_rule函数增加路由

In [ ]:
@app.route('/')
def hello():
    return "hello, world!"

### route装饰器
装饰器知识点：
* 把被装饰的函数替换成其他函数
* 在加载模块时立即执行
这里route是装饰器工厂函数，实现让装饰器接收其他参数（rule），然后返回装饰器decorator(f)
在装饰器中将rule和被装饰的函数名f._ _name_ _关联保存起来
将函数名和函数保存到self.view_functions字典中

In [ ]:
def route(self, rule, **options):    
    def decorator(f):
            self.add_url_rule(rule, f.__name__, **options)
            self.view_functions[f.__name__] = f
            return f
    return decorator

####  self.add_url_rule函数
函数参数知识点：
* 可变参数: * args 不传参数时为（）空元组
* 关键字参数: * * kvs 不传参数时为 {} 空字典

字典知识点：
* setdefault方法：如果键不存在于字典中，将会添加键并将值设为默认值，返回为该默认值。否则返回键对应的值。

self.url_map是一个Map对象存储所有的URL rules

In [ ]:
def add_url_rule(self, rule, endpoint, **options):
    options['endpoint'] = endpoint
    options.setdefault('methods', ('GET',))
    self.url_map.add(Rule(rule, **options))

### 处理请求的路由查找过程
#### 匹配请求
请求上下文中有属性url_adapter
 self.url_adapter = app.url_map.bind_to_environ(environ) 通过bind_to_environ提取environ字典中的信息，然后返回Map.bind，Map.bind中最后返回MapAdapter对象

In [ ]:
def match_request(self):
    """Matches the current request against the URL map and also
    stores the endpoint and view arguments on the request object
    is successful, otherwise the exception is stored.
    """
    rv = _request_ctx_stack.top.url_adapter.match()
    request.endpoint, request.view_args = rv
    return rv

#### MapAdapter.match方法
遍历map中的rules列表，查找匹配的rule。返回endpoint和参数（字典类型）

In [ ]:
def match(self, path_info=None, method=None, return_rule=False, query_args=None):
    """"""
    have_match_for = set()
    for rule in self.map._rules:
        try:
            rv = rule.match(path, method)
        except RequestSlash:
            raise RequestRedirect(self.make_redirect_url(
                url_quote(path_info, self.map.charset,
                          safe='/:|+') + '/', query_args))
        except RequestAliasRedirect as e:
            raise RequestRedirect(self.make_alias_redirect_url(
                path, rule.endpoint, e.matched_values, method, query_args))
        if rv is None:
            continue
        """"""

        if return_rule:
            return rule, rv
        else:
            return rule.endpoint, rv

        if have_match_for:
            raise MethodNotAllowed(valid_methods=list(have_match_for))
        raise NotFound()

#### 调用请求url绑定的函数
根据匹配返回的endpoint和参数，查找字典self.view_functions中endpoint对应的函数调用

In [ ]:
 def dispatch_request(self):
        """Does the request dispatching.  Matches the URL and returns the
        return value of the view or error handler.  This does not have to
        be a response object.  In order to convert the return value to a
        proper response object, call :func:`make_response`.
        """
        try:
            endpoint, values = self.match_request()
            return self.view_functions[endpoint](**values)
        except HTTPException, e:
            handler = self.error_handlers.get(e.code)
            if handler is None:
                return e
            return handler(e)
        except Exception, e:
            handler = self.error_handlers.get(500)
            if self.debug or handler is None:
                raise
            return handler(e)